In [30]:
#Installing and importing the required libraries
!pip install requests
from bs4 import BeautifulSoup 
import requests 



In [5]:
#GET request to the webserver to ddownload the HTML contents of a given webpage
page = requests.get("https://forecast.weather.gov/MapClick.php?lat=40.7146&lon=-74.0071#.Ye_PZerMKHs")

soup = BeautifulSoup(page.content, 'html.parser')
seven_day = soup.find(id="seven-day-forecast")
forecast_items = seven_day.find_all(class_="tombstone-container")
tonight = forecast_items[0]


In [6]:
#Prettify helps in looking at HTML structure and nested tags
print(tonight.prettify())

<div class="tombstone-container">
 <p class="period-name">
  Today
  <br/>
  <br/>
 </p>
 <p>
  <img alt="Today: Partly sunny, with a high near 44. Southwest wind 9 to 14 mph becoming northwest in the afternoon. " class="forecast-icon" src="newimages/medium/bkn.png" title="Today: Partly sunny, with a high near 44. Southwest wind 9 to 14 mph becoming northwest in the afternoon. "/>
 </p>
 <p class="short-desc">
  Partly Sunny
 </p>
 <p class="temp temp-high">
  High: 44 °F
 </p>
</div>


In [8]:
period = tonight.find(class_="period-name").get_text()
short_desc = tonight.find(class_="short-desc").get_text()
temp = tonight.find(class_="temp").get_text()
print(period)
print(short_desc)
print(temp)


Today
Partly Sunny
High: 44 °F


In [10]:
img = tonight.find("img")
desc = img['title']
print(desc)


Today: Partly sunny, with a high near 44. Southwest wind 9 to 14 mph becoming northwest in the afternoon. 


In [13]:
#By using developer tools and using inspect, we find each forecast item like Thursday Night etc. is contained in a div with the class tombstone - container
period_tags = seven_day.select(".tombstone-container .period-name")
periods = [pt.get_text()for pt in period_tags]
periods

['Today',
 'Tonight',
 'Wednesday',
 'WednesdayNight',
 'Thursday',
 'ThursdayNight',
 'Friday',
 'FridayNight',
 'Saturday']

In [16]:
short_descs = [sd.get_text() for sd in seven_day.select(".tombstone-container .short-desc")]
temps = [t.get_text() for t in seven_day.select(".tombstone-container .temp")]
descs = [d["title"] for d in seven_day.select(".tombstone-container img")]
print(short_descs)
print(temps)
print(descs)

['Partly Sunny', 'Mostly Clear', 'Sunny', 'Mostly Clear', 'Sunny', 'IncreasingClouds', 'Cloudy thenChance Snow', 'Chance Snow', 'Chance Snow']
['High: 44 °F', 'Low: 22 °F', 'High: 28 °F', 'Low: 18 °F', 'High: 31 °F', 'Low: 26 °F', 'High: 37 °F', 'Low: 27 °F', 'High: 31 °F']
['Today: Partly sunny, with a high near 44. Southwest wind 9 to 14 mph becoming northwest in the afternoon. ', 'Tonight: Mostly clear, with a low around 22. Wind chill values between 10 and 15. Northwest wind around 14 mph. ', 'Wednesday: Sunny, with a high near 28. Wind chill values between 10 and 15. Northwest wind around 14 mph. ', 'Wednesday Night: Mostly clear, with a low around 18. Wind chill values between 10 and 15. North wind 9 to 11 mph. ', 'Thursday: Sunny, with a high near 31. North wind 5 to 10 mph becoming south in the afternoon. ', 'Thursday Night: Increasing clouds, with a low around 26.', 'Friday: A 40 percent chance of snow after 1pm.  Cloudy, with a high near 37.', 'Friday Night: A 50 percent chan

In [21]:
import pandas as pd
weather = pd.DataFrame({
    "Period": periods,
    "Short Description": short_descs,
    "Temperature": temps,
    "Descriptions": descs
})
weather

,Period,Short Description,Temperature,Descriptions
0,Today,Partly Sunny,High: 44 °F,"Today: Partly sunny, with a high near 44. Sout..."
1,Tonight,Mostly Clear,Low: 22 °F,"Tonight: Mostly clear, with a low around 22. W..."
2,Wednesday,Sunny,High: 28 °F,"Wednesday: Sunny, with a high near 28. Wind ch..."
3,WednesdayNight,Mostly Clear,Low: 18 °F,"Wednesday Night: Mostly clear, with a low arou..."
4,Thursday,Sunny,High: 31 °F,"Thursday: Sunny, with a high near 31. North wi..."
5,ThursdayNight,IncreasingClouds,Low: 26 °F,"Thursday Night: Increasing clouds, with a low ..."
6,Friday,Cloudy thenChance Snow,High: 37 °F,Friday: A 40 percent chance of snow after 1pm....
7,FridayNight,Chance Snow,Low: 27 °F,Friday Night: A 50 percent chance of snow. Cl...
8,Saturday,Chance Snow,High: 31 °F,Saturday: A 40 percent chance of snow before 1...


In [24]:
temp_nums = weather["Temperature"].str.extract("(?P<temp_num>\d+)", expand=False)
weather["temp_num"] = temp_nums.astype('int')
temp_nums

0    44
1    22
2    28
3    18
4    31
5    26
6    37
7    27
8    31
Name: temp_num, dtype: object

In [26]:
#finding the mean of all of the high and low temperatures
weather["temp_num"].mean()

29.333333333333332

In [28]:
#selecting rows that happen at night
is_night = weather["Temperature"].str.contains("Low")
weather["is_night"] = is_night
is_night

0    False
1     True
2    False
3     True
4    False
5     True
6    False
7     True
8    False
Name: Temperature, dtype: bool

In [29]:
weather[is_night]

,Period,Short Description,Temperature,Descriptions,temp_num,is_night
1,Tonight,Mostly Clear,Low: 22 °F,"Tonight: Mostly clear, with a low around 22. W...",22,True
3,WednesdayNight,Mostly Clear,Low: 18 °F,"Wednesday Night: Mostly clear, with a low arou...",18,True
5,ThursdayNight,IncreasingClouds,Low: 26 °F,"Thursday Night: Increasing clouds, with a low ...",26,True
7,FridayNight,Chance Snow,Low: 27 °F,Friday Night: A 50 percent chance of snow. Cl...,27,True
